In [5]:
import rdkit
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from mordred import Calculator, descriptors
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


In [290]:
experimental_data = pd.read_excel('Data/Cp-Experimental-data.xlsx',index_col=0)
experimental_data

,Unnamed: 1,结构式,ΔfH°liquid/KJ/mol,3dsd
名称,,,,
nitroMethane,硝基甲烷,InChI=1S/CH3NO2/c1-2(3)4/h1H3,-113.,75-52-5
Carbon disulfide,二硫化碳,InChI=1S/CS2/c2-1-3,89.41,75-15-0
diiodoMethane,二碘甲烷,InChI=1S/CH2I2/c2-1-3/h1H2,68.5,75-11-6
methylHydrazine,甲基肼,"InChI=1S/CH6N2/c1-3-2/h3H,2H2,1H3",54.14,60-34-4
bromotrinitroMethane,溴三硝基甲烷,"InChI=1S/CBrN3O6/c2-1(3(6)7,4(8)9)5(10)11",48.5,560-95-2
...,...,...,...,...
"6,10,14-trimethyl-2-Pentadecanone","6,10,14-三甲基-2-十五烷酮",InChI=1S/C18H36O/c1-15(2)9-6-10-16(3)11-7-12-1...,-626.1,502-69-2
"(Z,Z)-9,12-Octadecadienoic acid","(Z,Z)-9,12-十八碳二烯酸",InChI=1S/C18H32O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,-634.7,60-33-3
Oleic Acid,油酸,InChI=1S/C18H34O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,-764.8,112-80-1


In [291]:
import os
folder_path = 'Data/Cp分子/'
files = []
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        files.append(filename)
print(files)

['79-06-1-3d.sdf', '79-09-4-3d.sdf', '51-79-6-3d.sdf', '55-63-0-3d.sdf', '56-81-5-3d.sdf', '57-10-3-3d.sdf', '57-11-4-3d.sdf', '57-14-7-3d.sdf', '57-55-6-3d.sdf', '57-57-8-3d.sdf', '59-89-2-3d.sdf', '60-33-3-3d.sdf', '60-34-4-3d.sdf', '61-90-5-3d.sdf', '64-17-5-3d.sdf', '64-19-7-3d.sdf', '64-67-5-3d.sdf', '65-71-4-3d.sdf', '67-56-1-3d.sdf', '67-63-0-3d.sdf', '67-64-1-3d.sdf', '67-68-5-3d.sdf', '67-71-0-3d.sdf', '68-12-2-3d.sdf', '70-29-1-3d.sdf', '71-23-8-3d.sdf', '71-36-3-3d.sdf', '71-41-0-3d.sdf', '71-43-2-3d.sdf', '71-55-6-3d.sdf', '74-83-9-3d.sdf', '74-88-4-3d.sdf', '74-89-5-3d.sdf', '74-93-1-3d.sdf', '74-96-4-3d.sdf', '74-98-6-3d.sdf', '75-00-3-3d.sdf', '75-01-4-3d.sdf', '75-03-6-3d.sdf', '75-04-7-3d.sdf', '75-05-8-3d.sdf', '75-07-0-3d.sdf', '75-08-1-3d.sdf', '75-11-6-3d.sdf', '75-12-7-3d.sdf', '75-15-0-3d.sdf', '75-21-8-3d.sdf', '75-26-3-3d.sdf', '75-30-9-3d.sdf', '75-31-0-3d.sdf', '75-34-3-3d.sdf', '75-35-4-3d.sdf', '75-36-5-3d.sdf', '75-50-3-3d.sdf', '75-52-5-3d.sdf', '75-56-9-

In [292]:
mols = []
fail = []
for i in files:
    suppl = Chem.SDMolSupplier(folder_path + str(i),strictParsing=False)
    if suppl[0] is not None:
        mols.append(suppl[0])
    else:
        fail.append(i)
mols
print("有%d个读取失败的,文件名称%s"%(len(fail),fail))

[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 56
[20:33:21] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:33:21] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:33:21] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:33:21] Warning: molecule is tagged as 3D, but all Z coords are zero
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 51
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 58
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 60
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 71
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 84
[20:33:21] CTAB version string invalid at line 4
[20:33:21] CTAB version string invalid at line 57
[20:33:21] CTAB ver

有1个读取失败的,文件名称['624-79-3-3d.sdf']


[20:33:22] CTAB version string invalid at line 61
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 88
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 67
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 72
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 72
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 88
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 88
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 88
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 83
[20:33:22] CTAB version string invalid at line 4
[20:33:22] CTAB version string invalid at line 60
[20:33:22] CTAB version string invalid at line 4
[20:33:22]

In [293]:
print("有%d个读取失败的,文件名称%s"%(len(fail),fail))
for i in fail:
    files.remove(i)
print("删除后有%d个分子"%(len(files)))

有1个读取失败的,文件名称['624-79-3-3d.sdf']
删除后有1468个分子


In [135]:
# 比较Rdkit与Mordred分子描述符 
calc = Calculator(descriptors, ignore_3D=True)
Mordred_description = []
Rdkit_description = [x[0] for x in Descriptors._descList]
for i in calc.descriptors:
    Mordred_description.append(i.__str__())
for i in Mordred_description:
    if i in Rdkit_description:
        j+=1
        Rdkit_description.remove(i)
        

55

In [136]:
# 比较Rdkit与Mordred分子描述符 
calc = Calculator(descriptors, ignore_3D=True)
Mordred_description = []
Rdkit_description = [x[0] for x in Descriptors._descList]
for i in calc.descriptors:
    Mordred_description.append(i.__str__())
for i in Mordred_description:
    if i in Rdkit_description:
        j+=1
        Rdkit_description.remove(i)
        
Molecular_descriptor = []

descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(Rdkit_description)
j =0
for i in mols:
    Calculator_descript = pd.DataFrame(calc.pandas([i]))
    rdkit_descriptors = pd.DataFrame([descriptor_calculator.CalcDescriptors(i)],columns=Rdkit_description)
    Calculator_descript = Calculator_descript.join(rdkit_descriptors)
    Molecular_descriptor.append(Calculator_descript)
    j+=1
    print(j)
    
a = Molecular_descriptor[0]
for i in Molecular_descriptor[1:]:
    a = a.append(i)
a = a.reset_index(drop=True)
a 

100%|██████████| 1/1 [00:00<00:00,  7.89it/s]


1


100%|██████████| 1/1 [00:00<00:00,  7.79it/s]


2


100%|██████████| 1/1 [00:00<00:00,  7.41it/s]


...

100%|██████████| 1/1 [00:00<00:00,  9.06it/s]


967


100%|██████████| 1/1 [00:00<00:00,  4.94it/s]


968


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,0.800000,2.444466,...,0,0,0,0,0,0,0,0,0,0
1,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,6.898979,1.149830,2.595800,...,0,0,0,0,0,0,0,0,0,0
2,3.535534,3.869735,0,0,6.987918,1.801938,3.603875,6.987918,1.164653,2.579830,...,0,0,0,0,0,0,0,0,0,0
3,2.449490,2.449490,0,0,3.464102,1.732051,3.464102,3.464102,0.866025,2.178059,...,0,0,0,0,0,0,0,0,0,0
4,3.047207,3.305183,0,0,5.226252,1.847759,3.695518,5.226252,1.045250,2.408576,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,11.532488,10.630940,0,1,19.925980,2.137643,4.275287,19.925980,1.172116,3.655749,...,0,0,0,0,0,0,0,0,4,0
964,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,8.054679,1.150668,2.739193,...,0,0,0,0,0,0,0,0,2,0
965,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,...,0,0,0,0,0,0,0,0,3,0
966,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,10.627503,1.180834,2.997414,...,0,0,0,0,0,0,0,0,4,0


In [238]:
a = Molecular_descriptor[0]
for i in Molecular_descriptor[1:]:
    a = a.append(i)
a = a.reset_index(drop=True)
a    

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,0.800000,2.444466,...,0,0,0,0,0,0,0,0,0,0
1,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,6.898979,1.149830,2.595800,...,0,0,0,0,0,0,0,0,0,0
2,3.535534,3.869735,0,0,6.987918,1.801938,3.603875,6.987918,1.164653,2.579830,...,0,0,0,0,0,0,0,0,0,0
3,2.449490,2.449490,0,0,3.464102,1.732051,3.464102,3.464102,0.866025,2.178059,...,0,0,0,0,0,0,0,0,0,0
4,3.047207,3.305183,0,0,5.226252,1.847759,3.695518,5.226252,1.045250,2.408576,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,11.532488,10.630940,0,1,19.925980,2.137643,4.275287,19.925980,1.172116,3.655749,...,0,0,0,0,0,0,0,0,4,0
964,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,8.054679,1.150668,2.739193,...,0,0,0,0,0,0,0,0,2,0
965,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,...,0,0,0,0,0,0,0,0,3,0
966,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,10.627503,1.180834,2.997414,...,0,0,0,0,0,0,0,0,4,0


In [1]:
# 将分子描述符与预测值结合
NAME = [x.replace("-3d","").replace(".sdf","") for x in files]
a.insert(0,"NAME",NAME)


NameError: name 'files' is not defined

In [240]:
a

,NAME,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,56-23-5,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,0.800000,...,0,0,0,0,0,0,0,0,0,0
1,56-81-5,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,6.898979,1.149830,...,0,0,0,0,0,0,0,0,0,0
2,57-06-7,3.535534,3.869735,0,0,6.987918,1.801938,3.603875,6.987918,1.164653,...,0,0,0,0,0,0,0,0,0,0
3,57-14-7,2.449490,2.449490,0,0,3.464102,1.732051,3.464102,3.464102,0.866025,...,0,0,0,0,0,0,0,0,0,0
4,57-55-6,3.047207,3.305183,0,0,5.226252,1.847759,3.695518,5.226252,1.045250,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,99916-30-0,11.532488,10.630940,0,1,19.925980,2.137643,4.275287,19.925980,1.172116,...,0,0,0,0,0,0,0,0,4,0
964,500005-27-6,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,8.054679,1.150668,...,0,0,0,0,0,0,0,0,2,0
965,500005-28-7,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,...,0,0,0,0,0,0,0,0,3,0
966,500005-29-8,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,10.627503,1.180834,...,0,0,0,0,0,0,0,0,4,0


In [217]:
import math

# lef_series_index = [x.replace("\xa0","") for x in experimental_data["3dSD"].values]
y_series_index = []
for x in experimental_data["3dSD"].values:
    if type(x) == str:
        y_series_index.append(x.replace("\xa0",""))
    else:
        y_series_index.append(x)
        
for i in range(len(y_series_index)):
    if type(y_series_index[i])!=str:
        y_series_index[i] = y_series_index[i-1]
y_series_value = [float(x) for x in experimental_data["Cp,liquid (J/mol*K)"].values] 
y_series = pd.Series(y_series_value)
y_series.index = y_series_index
y_series = pd.DataFrame(y_series)
y_series

,0
74-88-4,82.75
74-88-4,82.68
67-66-3,114.25
67-66-3,113.40
67-66-3,114.00
...,...
7098-26-2,1595.00
555-44-2,1665.00
555-44-2,1753.10
555-43-1,1975.00


In [252]:
final_data = pd.DataFrame()
for i in a["NAME"]:
    temp_data = pd.DataFrame()
    for j in range(y_series[y_series.index == i].shape[0]):
        temp_data = temp_data.append(a[a["NAME"] ==i])
    temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
    final_data = final_data.append(temp_data)
        
    

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

...

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_data.insert(1,"Cp",y_series[y_series.index ==i].values)
/tmp/ipykernel_242379/2336239764.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

In [253]:
final_data

,NAME,Cp,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,56-23-5,131.3,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,133.1,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,130.5,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,131.5,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
1,56-81-5,218.9,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,6.898979,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,500005-28-7,248.9,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,8.054679,...,0,0,0,0,0,0,0,0,2,0
965,500005-29-8,282.0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,...,0,0,0,0,0,0,0,0,3,0
966,500005-57-2,431.0,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,10.627503,...,0,0,0,0,0,0,0,0,4,0
967,60-01-5,555.3,19.557227,19.999011,0,0,25.933953,2.744325,5.398836,25.933953,...,0,0,0,0,0,0,0,0,0,0


In [276]:
temperture_series =pd.Series(experimental_data["Temperature (K)"])
temperture_series.index = y_series_index
temperture_value = []
for i in a["NAME"]:
    for j in temperture_series[temperture_series.index == i].values:
        temperture_value.append(j)
temperture_value

[298.15,
 301.2,
 303.3,
 293.0,
 298.15,
 313.15,
 301.2,
 299.4,
 289.7,
 293.15,
 290.0,
 298.15,
 298.0,
 303.0,
 276.7,
 298.15,
 298.15,
 298.15,
 293.0,
 290.0,
 255.2,
 308.0,
 286.6,
 298.15,
 298.0,
 293.0,
 323.0,
 267.3,
 288.0,
 298.15,
 298.15,
 298.15,
 288.15,
 313.2,
 250.0,
 297.359,
 316.0,
 270.0,
 303.0,
 298.15,
 290.0,
 291.5,
 298.15,
 332.0,
 297.1,
 413.0,
 298.15,
 298.15,
 313.2,
 311.0,
 323.0,
 270.0,
 292.0,
 298.15,
 311.6,
 324.0,
 293.1,
 298.15,
 296.99,
 283.0,
 302.4,
 298.15,
 300.0,
 303.2,
 298.15,
 298.15,
 308.0,
 298.0,
 298.0,
 303.4,
 313.2,
 270.0,
 298.15,
 321.05,
 293.15,
 301.2,
 303.5,
 298.15,
 301.26,
 313.2,
 298.15,
 293.15,
 322.05,
 303.15,
 300.0,
 316.0,
 287.8,
 323.15,
 298.15,
 257.29,
 299.59,
 283.0,
 280.0,
 100.0,
 94.0,
 100.32,
 180.0,
 200.0,
 170.0,
 293.15,
 249.67,
 298.0,
 298.15,
 300.0,
 259.28,
 280.0,
 298.15,
 300.0,
 298.15,
 298.0,
 230.0,
 300.0,
 290.0,
 298.0,
 298.15,
 298.15,
 298.0,
 298.15,
 303.15,


In [278]:
final_data.insert(2,"Temperature",temperture_value)
final_data = final_data.reset_index(drop=True)
final_data

,NAME,Cp,Temperature,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,56-23-5,131.3,298.15,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,133.1,301.20,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,130.5,303.30,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
0,56-23-5,131.5,293.00,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
1,56-81-5,218.9,298.15,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,500005-28-7,248.9,298.15,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,...,0,0,0,0,0,0,0,0,2,0
965,500005-29-8,282.0,298.15,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,...,0,0,0,0,0,0,0,0,3,0
966,500005-57-2,431.0,298.00,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,...,0,0,0,0,0,0,0,0,4,0
967,60-01-5,555.3,298.15,19.557227,19.999011,0,0,25.933953,2.744325,5.398836,...,0,0,0,0,0,0,0,0,0,0


In [281]:
final_data = final_data.reset_index(drop=True)
final_data

,NAME,Cp,Temperature,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,56-23-5,131.3,298.15,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
1,56-23-5,133.1,301.20,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
2,56-23-5,130.5,303.30,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
3,56-23-5,131.5,293.00,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,...,0,0,0,0,0,0,0,0,0,0
4,56-81-5,218.9,298.15,3.644924,4.097495,0,0,6.898979,1.931852,3.863703,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,500005-28-7,248.9,298.15,4.242641,4.530950,0,0,8.054679,1.847759,3.695518,...,0,0,0,0,0,0,0,0,2,0
1394,500005-29-8,282.0,298.15,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,...,0,0,0,0,0,0,0,0,3,0
1395,500005-57-2,431.0,298.00,5.656854,5.715477,0,0,10.627503,1.902113,3.804226,...,0,0,0,0,0,0,0,0,4,0
1396,60-01-5,555.3,298.15,19.557227,19.999011,0,0,25.933953,2.744325,5.398836,...,0,0,0,0,0,0,0,0,0,0


In [282]:
final_data.to_excel("Data/Cp_Rdkit-1398-1767.xlsx")